In [1]:
import pandas as pd
import geopandas as gpd
import dask.dataframe as dd
import dask.distributed
from shapely.geometry import Point, Polygon
from scipy.spatial import cKDTree

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pd.set_option('display.max_rows',10)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)

In [4]:
gdf_osyu_100m_mesh = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/gdf_osyu_100m_mesh.shp').filter(['Meshcode','longitude', 'latitude',  'geometry'])

In [5]:
gdf_railway = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/鉄道/N02-22_RailroadSection.shp', encoding='shift-jis') #このデータは道路中心線

In [6]:
gdf_railway

,N02_001,N02_002,N02_003,N02_004,geometry
0,23,5,沖縄都市モノレール線,沖縄都市モノレール,"LINESTRING (127.67948 26.21454, 127.67970 26.21474, 127.67975 26.21480, 127.68217 26.21728, 127.68357 26.21862, 127.68394 26.21891, 127.68419 26.21905)"
1,12,5,いわて銀河鉄道線,アイジーアールいわて銀河鉄道,"LINESTRING (141.29139 40.33740, 141.29176 40.33723, 141.29243 40.33692, 141.29323 40.33654, 141.29379 40.33624, 141.29411 40.33608, 141.29490 40.33563, 141.29624 40.33477, 141.29813 40.33354, 141.29862 40.33317)"
2,12,5,いわて銀河鉄道線,アイジーアールいわて銀河鉄道,"LINESTRING (141.27554 40.23936, 141.27567 40.23884, 141.27587 40.23827, 141.27622 40.23756, 141.27656 40.23694, 141.27722 40.23579, 141.27789 40.23462, 141.27856 40.23344, 141.27922 40.23236, 141.27983 40.23153, 141.28006 40.23123, 141.28011 40.23118, 141.28034 40.23094, 141.28092 40.23042, 141.28207 40.22963, 141.28328 40.22883, 141.28345 40.22874)"
3,12,5,いわて銀河鉄道線,アイジーアールいわて銀河鉄道,"LINESTRING (141.28659 40.26092, 141.28538 40.25874)"
4,12,5,いわて銀河鉄道線,アイジーアールいわて銀河鉄道,"LINESTRING (141.29082 40.28615, 141.29089 40.28582, 141.29103 40.28485)"
...,...,...,...,...,...
21911,12,5,あいの風とやま鉄道線,あいの風とやま鉄道,"LINESTRING (137.24023 36.71916, 137.24056 36.71999)"
21912,12,5,あいの風とやま鉄道線,あいの風とやま鉄道,"LINESTRING (137.24056 36.71999, 137.24071 36.72035, 137.24116 36.72148, 137.24182 36.72320, 137.24206 36.72381, 137.24235 36.72457, 137.24286 36.72585, 137.24308 36.72641, 137.24318 36.72667, 137.24331 36.72702, 137.24382 36.72829, 137.24396 36.72865, 137.24447 36.72995, 137.24479 36.73077, 137.24540 36.73230, 137.24558 36.73279, 137.24623 36.73442, 137.24652 36.73516, 137.24698 36.73627, 137.24776 36.73831, 137.24834 36.73973)"
21913,11,2,宗谷線,北海道旅客鉄道,"LINESTRING (142.46548 44.33083, 142.46548 44.33151)"
21914,11,2,札沼線,北海道旅客鉄道,"LINESTRING (141.44875 43.18960, 141.44622 43.18756, 141.44481 43.18642, 141.44369 43.18551, 141.44097 43.18335, 141.43831 43.18119, 141.43707 43.18019)"


In [7]:
#座標系を変更
src_proj = 6668 # 変換前の座標系を指定
dst_proj = 6678  # 変換後の座標系を指定
# ポイント（ダムデータ、TransformPointの引数は緯度,経度の順番で指定）
gdf_railway.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_railway = gdf_railway.to_crs(epsg=dst_proj)  # 変換後座標に変換

In [8]:
#線路のデータフレームのカラム名を変換
gdf_railway.rename(columns={'N02_001': 'Railway_classification', 'N02_002': 'Railway_Business_Type', 'N02_003': 'Railway_route_name', 'N02_004':'Railway_operating_company'}, inplace=True)

In [9]:
#重心座標のPointを100mメッシュのデータフレームに追加
gdf_osyu_100m_mesh['center_geometry'] = [Point(x) for x in zip(gdf_osyu_100m_mesh['longitude'], gdf_osyu_100m_mesh['latitude'])]

In [10]:
# cKDTreeを使用して最近傍と距離を求める
line_coords = gdf_railway.geometry.apply(lambda geom: list(geom.coords[0]))
point_coords = gdf_osyu_100m_mesh.center_geometry.apply(lambda geom: list(geom.coords[0]))

line_tree = cKDTree(line_coords.tolist())
point_tree = cKDTree(point_coords.tolist())

# 最近傍の検索
distances, line_indices = line_tree.query(point_coords.tolist(), k=1)
nearest_lines = gdf_railway.iloc[line_indices]

# 距離を結果に追加
nearest_lines['distance_railway'] = distances

# 結果を表示
print(nearest_lines)

#lineのgeometryxはいらないので削除
nearest_lines.drop('geometry', axis=1, inplace=True)

#100mメッシュに対して同じ高速道路が最近傍だとindexが複数できてしまうので並び替え
nearest_lines.reset_index(drop=True, inplace=True)

#100mメッシュのデータフレームに最近棒の高速道路を追加
gdf_railway = gdf_osyu_100m_mesh.join(nearest_lines)

      Railway_classification Railway_Business_Type Railway_route_name  \
13970                     11                     2                花輪線   
13970                     11                     2                花輪線   
13970                     11                     2                花輪線   
13970                     11                     2                花輪線   
13970                     11                     2                花輪線   
...                      ...                   ...                ...   
13970                     11                     2                花輪線   
13970                     11                     2                花輪線   
13970                     11                     2                花輪線   
13970                     11                     2                花輪線   
13970                     11                     2                花輪線   

      Railway_operating_company  \
13970                   東日本旅客鉄道   
13970                   東日本旅客鉄道   
13970             

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
<ipython-input-10-0f7b94120d19>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nearest_lines.drop('geometry', axis=1, inplace=True)


In [11]:
#座標系を変更
src_proj = 6678 # 変換前の座標系を指定
dst_proj = 4326  # 変換後の座標系を指定
# ポイント（ダムデータ、TransformPointの引数は緯度,経度の順番で指定）
gdf_railway.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_railway = gdf_railway.to_crs(epsg=dst_proj)  # 変換後座標に変換

In [13]:
# #geometryが二つあると出力できないのでcenter_geometryを削除
gdf_railway.drop(columns=['longitude', 'latitude', 'geometry', 'center_geometry'], axis=1, inplace=True)

In [14]:
gdf_railway.to_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市 最近傍鉄道/奥州市_最近傍線路.csv', encoding='shift-jis')